In [36]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# ChromeDriver 路径
driver_path = r"C:\chromedriver-win64\chromedriver.exe"
options = Options()
options.add_argument('--incognito')

# 初始化 WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 打开目标网页
url = "https://www.starbucks.com.tw/stores/storesearch.jspx"
driver.get(url)

# 等待页面加载
time.sleep(3)

# 存储数据
store_data = []

try:
    while True:  # 直到页面没有更多数据
        # 等待门市数据加载
        store_list = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.store_data"))
        )

        # 遍历当前页面的门市数据
        for store in store_list:
            store_name = store.find_element(By.CSS_SELECTOR, "h4.store_name").text.strip()
            address = store.find_element(By.CSS_SELECTOR, "p.store_add").text.strip()

            # 从 onmouseover 中提取座标并合并为纬度和经度
            onmouseover_data = store.get_attribute("onmouseover")
            coordinates = onmouseover_data.split('(')[1].split(')')[0].split(',')
            latitude = coordinates[1].strip()  # 纬度是第二位
            longitude = coordinates[2].strip()  # 经度是第三位
            coordinates_combined = f"({latitude}, {longitude})"  # 合并成一个字段

            # 点击进入详细信息页面
            button = store.find_element(By.XPATH, ".//a[@class='btn info']")
            button.click()
            time.sleep(2)

            # 获取附加服务
            additional_services = []
            try:
                service_elements = driver.find_elements(By.CSS_SELECTOR, "ul.icon_list li p")
                for service in service_elements:
                    service_text = service.text.strip()
                    if service_text:  # 只添加非空的服务
                        additional_services.append(service_text)
            except Exception as e:
                print(f"附加服务获取失败: {e}")
            additional_services = ", ".join(additional_services)

            # 获取营业时间
            try:
                business_hours = driver.find_element(By.CSS_SELECTOR, "p.store_phone + p").text.strip()
            except Exception as e:
                business_hours = "未找到营业时间"
                print(f"营业时间获取失败: {e}")

            # 获取电话号码
            try:
                store_phone = driver.find_element(By.CSS_SELECTOR, "p.store_phone a").text.strip()
            except Exception as e:
                store_phone = "未找到电话"
                print(f"电话获取失败: {e}")

            # 输出抓取的数据
            print(f"门市名称: {store_name}")
            print(f"电话: {store_phone}")
            print(f"地址: {address}")
            print(f"营业时间: {business_hours}")
            print(f"座标: {coordinates_combined}")
            print(f"附加服务: {additional_services}")
            print("=" * 40)  # 分隔线

            # 存储抓取到的数据
            store_data.append([store_name, store_phone, address, business_hours, coordinates_combined, additional_services])

            # 返回主页面
            close_button = driver.find_element(By.CSS_SELECTOR, "a.btn.close")
            close_button.click()
            time.sleep(1)

        # 向下滚动页面以加载更多数据
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        # 检查是否能加载更多数据（如果没有新的数据加载，停止）
        new_store_list = driver.find_elements(By.CSS_SELECTOR, "li.store_data")
        if len(new_store_list) == len(store_data):
            break

except Exception as e:
    print(f"爬取过程中发生错误：{e}")
finally:
    # 保存到 CSV 文件
    with open("starbucks_store.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["门市名称", "电话", "地址", "营业时间", "座标", "附加服务"])
        writer.writerows(store_data)

    driver.quit()


门市名称: 土城大潤發門市
电话: 02-22697625
地址: 新北市土城區永安街25號1F
营业时间: (一~五)10:00~21:30 (六~日)09:30~21:30
座标: (24.963868, 121.433817)
附加服务: Nitro Cold Brew 氮氣冷萃咖啡, Mobile Order & Pay 行動預點, Pickup 到店自取, Dine-In Ordering 內用點餐, Borrow & Return a Cup 循環杯租借/歸還服務, Starbucks Wi-Fi 無線上網
门市名称: 學勤北大門市
电话: 
地址: 新北市樹林區大雅路288號
营业时间: 
座标: (24.947724, 121.379658)
附加服务: Nitro Cold Brew 氮氣冷萃咖啡, Mobile Order & Pay 行動預點, Pickup 到店自取, Dine-In Ordering 內用點餐, Borrow & Return a Cup 循環杯租借/歸還服務, Starbucks Wi-Fi 無線上網
门市名称: 樹林中山門市
电话: 
地址: 新北市樹林區中山路一段157號 (近立人街口)
营业时间: 
座标: (24.98881, 121.422057)
附加服务: Mobile Order & Pay 行動預點, Pickup 到店自取, Dine-In Ordering 內用點餐, Borrow & Return a Cup 循環杯租借/歸還服務, Starbucks Wi-Fi 無線上網, Coffee Voucher 紙本飲料劵 / Beverage eCard 星運卡
门市名称: 三峽學成門市
电话: 
地址: 新北市三峽區學成路317號
营业时间: 
座标: (24.944194, 121.377258)
附加服务: Mobile Order & Pay 行動預點, Pickup 到店自取, Dine-In Ordering 內用點餐, Borrow & Return a Cup 循環杯租借/歸還服務, Starbucks Wi-Fi 無線上網
门市名称: 土城日月光門市
电话: 
地址: 新北市土城區學府路二段210號1F
营业时间: 
座标: (24.978648, 121.444178)
附加服务: 

In [26]:
import csv
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

# 初始化 WebDriver
driver_path = r"C:\chromedriver-win64\chromedriver.exe"  # 修改為您的 ChromeDriver 路徑
options = Options()
options.add_argument('--incognito')

# 啟動 WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 輸入與輸出 CSV 文件
input_csv = "starbucks_store.csv"  # 修改為您的輸入 CSV 文件名稱
output_csv = "starbucks_google_maps_data.csv"  # 輸出的結果文件名稱
store_data = []

def get_business_hours(driver):
    try:
        # 等待營業時間元素加載完成
        time_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@aria-label, '隱藏本週營業時間')]"))
        )
        aria_label = time_element.get_attribute("aria-label")
        
        # 提取整週營業時間
        week_hours = {}
        for day in ["星期一", "星期二", "星期三", "星期四", "星期五", "星期六", "星期日"]:
            start_index = aria_label.find(day)
            if start_index != -1:
                end_index = aria_label.find("星期", start_index + 1)  # 下個星期的開始位置
                if end_index == -1:
                    end_index = len(aria_label)
                day_hours = aria_label[start_index:end_index].strip()
                week_hours[day] = day_hours
        return week_hours
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        return None

def search_store(driver, search_textbox, store_name, store_address):
    try:
        # 清除搜索欄位
        clear_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.lSDxNd button[aria-label='關閉']"))
        )
        clear_button.click()
        time.sleep(random.uniform(1, 2))
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        pass

    # 搜索門市名稱與地址
    search_textbox.send_keys(f"星巴克 {store_name} {store_address}")
    search_button = driver.find_element(By.ID, "searchbox-searchbutton")
    search_button.click()

def main():
    driver.get("https://www.google.com/maps")
    time.sleep(10)

    data_list = []

    # 讀取 CSV 檔案中的店名和地址
    with open(input_csv, "r", encoding="utf-8-sig") as f:
        reader = csv.reader(f)
        next(reader)  # 跳過表頭
        name_address_info = [(row[0], row[1]) for row in reader]  # 假設店名在第一欄，地址在第二欄

    search_textbox = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input#searchboxinput"))
    )

    for index, (name, address) in enumerate(name_address_info, start=1):
        print(f"\n========== 店家 {index} ==========")
        search_store(driver, search_textbox, name, address)
        time.sleep(random.uniform(5, 10))

        # 獲取店名
        try:
            store_name_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf.lfPIob"))
            )
            store_name = store_name_element.text
        except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
            print(f"找不到店家資訊: {name}")
            continue

        print(f"店名： {store_name}")

        # 獲取營業時間
        week_hours = get_business_hours(driver)
        if week_hours:
            print("整週營業時間：")
            for day, hours in week_hours.items():
                print(f"{day}: {hours}")
        else:
            print("無法提取營業時間")
            continue

        # 保存資料
        data_list.append([store_name, *week_hours.values()])

    # 關閉瀏覽器
    driver.quit()

    # 儲存結果至 CSV
    with open(output_csv, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["店名", "星期一", "星期二", "星期三", "星期四", "星期五", "星期六", "星期日"])
        writer.writerows(data_list)

    print("\n========== 爬取結果 ==========")
    for data in data_list:
        print(data)

if __name__ == "__main__":
    main()



========== 店家 1 ==========
店名： 星巴克土城大潤發門市
整週營業時間：
星期一: 星期一、10:00 到 21:30;
星期二: 星期二、10:00 到 21:30. 隱藏本週營業時間
星期三: 星期三、10:00 到 21:30;
星期四: 星期四、10:00 到 21:30;
星期五: 星期五、10:00 到 21:30;
星期六: 星期六、09:30 到 21:30;
星期日: 星期日、09:30 到 21:30;

========== 店家 2 ==========
店名： STARBUCKS 星巴克 (學勤北大門市)
整週營業時間：
星期一: 星期一、06:30 到 21:30;
星期二: 星期二、06:30 到 21:30. 隱藏本週營業時間
星期三: 星期三、06:30 到 21:30;
星期四: 星期四、06:30 到 21:30;
星期五: 星期五、06:30 到 21:30;
星期六: 星期六、06:30 到 21:30;
星期日: 星期日、06:30 到 21:30;

========== 店家 3 ==========
店名： 星巴克 樹林中山門市
整週營業時間：
星期一: 星期一、07:00 到 21:00;
星期二: 星期二、07:00 到 21:00. 隱藏本週營業時間
星期三: 星期三、07:00 到 21:00;
星期四: 星期四、07:00 到 21:00;
星期五: 星期五、07:00 到 21:00;
星期六: 星期六、07:00 到 21:00;
星期日: 星期日、07:00 到 21:00;

========== 店家 4 ==========
店名： STARBUCKS 星巴克 (三峽學成門市)
整週營業時間：
星期一: 星期一、07:00 到 21:30;
星期二: 星期二、07:00 到 21:30. 隱藏本週營業時間
星期三: 星期三、07:00 到 21:30;
星期四: 星期四、07:00 到 21:30;
星期五: 星期五、07:00 到 21:30;
星期六: 星期六、07:00 到 21:30;
星期日: 星期日、07:00 到 21:30;

========== 店家 5 ==========
店名： STARBUCKS 星巴克 (土城日月光門市)
整週營業時間：
